<a href="https://colab.research.google.com/github/AnkitaSK/NamasteBot/blob/main/NamasteBot_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 48.7 MB/s eta 0:0

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 5.9 MB/s eta 0:00:00


In [1]:
!pip uninstall langchain-google
!pip install --upgrade langchain-google-genai

  Using cached langchain_google_genai-2.0.9-py3-none-any.whl.metadata (3.6 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached langchain_google_genai-2.0.9-py3-none-any.whl (41 kB)


In [2]:
pip show langchain-google-genai


Name: langchain-google-genai
Version: 2.0.9
Summary: An integration package connecting Google's genai package and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: filetype, google-generativeai, langchain-core, pydantic
Required-by: 


In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import google.generativeai as genai
import os

In [21]:
from google.colab import userdata

In [23]:
# Configure Google Gemini API with LangChain
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
llm = GoogleGenerativeAI(model="gemini-1.5-pro-latest", google_api_key=GOOGLE_API_KEY)
print("GoogleGenerativeAI import successful!")


GoogleGenerativeAI import successful!


In [25]:
# Load and Preprocess PDFs using LangChain
loader = PyPDFLoader("Haridwar-Travel-Guide-by-ixigo.com.pdf")
documents = loader.load()

# Use RecursiveCharacterTextSplitter for efficient chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


In [26]:
# Create Vector Store with LangChain's Chroma Wrapper
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [27]:
# Create a RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # or "map_reduce" for longer contexts
    retriever=retriever,
    return_source_documents=True
)


In [9]:
!pip install langdetect
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=24fe6031e30a26ac2a531ed974f64d4aa5a2c96a57b0ed6ed0cf368a0514a74e
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [28]:
# Implement Multilingual Query Handling
from langdetect import detect
from deep_translator import GoogleTranslator

def detect_language(text):
    try:
        return detect(text)
    except:
        return "en"

def translate_to_english(text, lang):
    if lang != "en":
        return GoogleTranslator(source=lang, target="en").translate(text)
    return text

def translate_back(text, lang):
    if lang != "en":
        return GoogleTranslator(source="en", target=lang).translate(text)
    return text

async def rag_pipeline_with_translation(query):
    detected_lang = detect_language(query)
    translated_query = translate_to_english(query, detected_lang)

    response = qa_chain.invoke(translated_query)  # Use `.invoke()` instead of `.run()`

    result_text = response["result"]  # Extract only the result
    return translate_back(result_text, detected_lang)


In [29]:
query = "गंगा नदी के पास एक पवित्र शहर के बारे में बताएं"
response = await rag_pipeline_with_translation(query)  # Call the function

# Extracting 'result' safely
if isinstance(response, dict):
    print(response.get("result", "No result found"))
else:
    print(response)  # Print full response for debugging



हरिद्वार गंगा नदी के पास स्थित एक पवित्र शहर है। लोगों का मानना ​​है कि यहाँ नदी में स्नान करने से पाप धुल जाते हैं। यहाँ का एक प्रमुख स्थान हर की पौड़ी है, जहाँ लोग पवित्र स्नान करते हैं। यहाँ चंडी देवी मंदिर, माया देवी मंदिर और मनसा देवी मंदिर जैसे मंदिर भी हैं।


In [31]:
# Example usage1 English
query = "Tell about a holy city near river Ganga."
response = await rag_pipeline_with_translation(query)
print(response)

Haridwar is a holy city located on the banks of the river Ganges.  People believe that bathing in the river there washes away sins.  A key location in the city is Har ki Pauri, where people take holy dips.  Several temples, such as Chandi Devi Temple, Maya Devi Temple, and Mansa Devi Temple are also located in the city.

